In [ ]:
from dagster import EnvVar
from pyiceberg.catalog import Catalog, load_catalog

name = "default"
warehouse = EnvVar("NESSIE_WAREHOUSE").get_value()
branch = EnvVar("NESSIE_BRANCH").get_value()
uri = EnvVar("NESSIE_URI").get_value()
py_io_impl = "pyiceberg.io.pyarrow.PyArrowFileIO"
s3_endpoint = EnvVar("DESTINATION__FILESYSTEM__CREDENTIALS__AWS_S3_ENDPOINT").get_value()
s3_access_key_id = EnvVar(
    "DESTINATION__FILESYSTEM__CREDENTIALS__AWS_ACCESS_KEY_ID"
).get_value()
s3_secret_access_key = EnvVar(
    "DESTINATION__FILESYSTEM__CREDENTIALS__AWS_SECRET_ACCESS_KEY"
).get_value()
catalog_type = "rest"
catalog = load_catalog(
            **{
                "warehouse": warehouse,
                "uri": f"{uri}/{branch}",
                "py-io-impl": py_io_impl,
                "s3.endpoint": s3_endpoint,
                "s3.access-key-id": s3_access_key_id,
                "s3.secret-access-key": s3_secret_access_key,
                "type": catalog_type,
            },
        )

print(branch)

In [20]:
catalog.list_tables("staging")

[]

In [19]:
# table = catalog.drop_table("staging.cit0day_password_files")

In [8]:
table = catalog.load_table("staging.cit0day_password_files")

In [12]:
con = table.scan(limit=50).to_duckdb(table_name="cit0day_password_files")
print(pl.DataFrame(con.execute(
        "select * from cit0day_password_files"
    ).fetch_arrow_table()))

shape: (50, 6)
┌─────────────────┬─────────────────┬─────────────────┬─────────────────┬────────┬─────────────────┐
│ email           ┆ username        ┆ domain          ┆ data            ┆ bucket ┆ prefix          │
│ ---             ┆ ---             ┆ ---             ┆ ---             ┆ ---    ┆ ---             │
│ str             ┆ str             ┆ str             ┆ str             ┆ str    ┆ str             │
╞═════════════════╪═════════════════╪═════════════════╪═════════════════╪════════╪═════════════════╡
│ 0006diana06love ┆ 0006diana06love ┆ ya.ru:a98ce02c9 ┆ a98ce02c9b0077f ┆ raw    ┆ extracted/Cit0d │
│ 06@ya.ru        ┆ 06              ┆ b0077fa618ae812 ┆ a618ae812b80698 ┆        ┆ ay Prem         │
│                 ┆                 ┆ …               ┆ …               ┆        ┆ [_speci…        │
│ 0079817235269@y ┆ 0079817235269   ┆ ya.ru:91b40499a ┆ 91b40499a08f6d6 ┆ raw    ┆ extracted/Cit0d │
│ a.ru            ┆                 ┆ 08f6d6f689c113b ┆ f689c113bf43056 ┆   

In [52]:
from pyiceberg.partitioning import PartitionSpec, PartitionField
from pyiceberg.transforms import DayTransform

In [53]:
from pyiceberg import transforms

In [10]:
import polars as pl